In [107]:
#Produces some data that allows you to test the interactive chart without using data
from faker import Faker
fake = Faker()

In [108]:
import json
with open('random_images.json') as data_file:    
    random_images = json.load(data_file)

In [109]:
import requests
import json
import random

def create_node():
    node = {}
    node["full_name"] = fake.name()
    node["email_address"] = fake.email()
    node["phone_numbers"] = fake.phone_number()
    node["id"] = fake.md5()
    node["summary"] = fake.text()
    node["job_title"] = fake.job()
    node["web_url"] = fake.url()
    
    node["colour"] = fake.safe_hex_color()
    
    node["statistic_1"] = fake.random_number(3)
    node["statistic_2"] = fake.random_number(1)
    
    node["mugshot_url_template"] = "http://lorempixel.com/200/200"
    #If you have access to a list of images you can use this:
    node["mugshot_url_template"] = random.choice(random_images)["image_urls"]["epic"]
    
    

    return node
root = create_node()

In [110]:
root = create_node()
import numpy as np
import random

from scipy import stats

def create_random_children(node,depth):
    
    num_children = np.random.binomial(4-depth, 0.6, 1)[0]
    
    node["children"] = []
    
    for i in range(num_children):
        this_node = create_node()
        node["children"].append(this_node)
    
    for node in node["children"]:
        create_random_children(node, depth+1)
        
create_random_children(root,1)

In [111]:

#ALso need data that for the select box
select_box = []


def add_to_select_box(node):

    new_item = {}
    new_item["id"] = node["id"]
    new_item["text"] =  node["full_name"] + " " +node["job_title"]
    select_box.append(new_item)
    
    if "children" in node:
        for c in node["children"]:
            add_to_select_box(c)


#Iterate through the nodes, adding to select box

add_to_select_box(root)    
select_box.sort(key=lambda x: x["text"])

In [112]:
final_data = {"tree": root, "select_box":select_box}
    
import json
with open('../website/data/random_data.json', 'w') as outfile:
    json.dump(final_data, outfile)

In [113]:
flat = []

def flatten(tree):
    """
    Recursively iterate through tree adding rows to the flat array
    """

    def next_node(node, parent_id):
        
        this_row = {}
        
        this_row["parent"] = parent_id
        
        this_row["full_name"] = node["full_name"]
        this_row["email_address"] = node["email_address"]
        this_row["phone_numbers"] = node["phone_numbers"]
        this_row["id"] = node["id"]
        this_row["summary"] = node["summary"]
        this_row["job_title"] = node["job_title"]
        this_row["web_url"] = node["web_url"]
        this_row["colour"] = node["colour"]
        this_row["mugshot_url_template"] = node["mugshot_url_template"]
        
        this_row["statistic_1"] = node["statistic_1"] 
        this_row["statistic_2"] = node["statistic_2"] 
        
        flat.append(this_row)
        
        for child in node["children"]:
            next_node(child, node["id"])
    
    next_node(tree, "")
    
flatten(root)
        
import pandas as pd
df = pd.DataFrame(flat)
df.to_csv("random_data_flat_file.csv", encoding="utf-8", index=False)
df

,colour,email_address,full_name,id,job_title,mugshot_url_template,parent,phone_numbers,statistic_1,statistic_2,summary,web_url
0,#55dd00,misti95@champlinsauer.com,Ms. Jean Yundt PhD,ecbaee24da9c6ae57bd195bb53a8c157,Financial adviser,https://s3.amazonaws.com/uifaces/faces/twitter...,,684-037-3445,841,5,Libero inventore totam voluptatem. Ratione et ...,http://rutherford.info/
1,#112200,edooley@kundemante.net,Ms. Belinda Yundt DVM,9b96247538f35dbe04e86200ebb4908b,Education administrator,https://s3.amazonaws.com/uifaces/faces/twitter...,ecbaee24da9c6ae57bd195bb53a8c157,1-491-097-6914,228,0,Cupiditate voluptatem aut sit voluptatibus nos...,http://www.rodriguezsauer.com/
2,#bbbb00,emmerich.kiefer@zulauf.com,Ayleen Von PhD,15a987d0c84833ca4bfe61c77dae1fc0,Broadcast journalist,https://s3.amazonaws.com/uifaces/faces/twitter...,9b96247538f35dbe04e86200ebb4908b,1-667-335-6423x09357,342,8,Consequuntur possimus repellat praesentium fug...,http://www.mclaughlin.info/
3,#22bb00,ywillms@kris.com,Kaeden Altenwerth,491ee869a19bd81bbe9fb39dc924d238,Podiatrist,https://s3.amazonaws.com/uifaces/faces/twitter...,15a987d0c84833ca4bfe61c77dae1fc0,(919)438-3184x2684,406,1,Numquam unde fuga culpa commodi repudiandae mo...,http://www.osinskikuphal.com/
4,#44cc00,henderson.daugherty@gmail.com,Koby Lind,da7c474ce0cda9e6591a6e5c7dcefe42,Press photographer,https://s3.amazonaws.com/uifaces/faces/twitter...,ecbaee24da9c6ae57bd195bb53a8c157,1-858-183-5849x3662,323,2,Sint et aspernatur accusantium doloribus fuga ...,http://www.keebler.com/
5,#ff1100,uosinski@hotmail.com,Oley Kuhic,cf27768cffb96d72b40b2a2801164a40,Drilling engineer,https://s3.amazonaws.com/uifaces/faces/twitter...,da7c474ce0cda9e6591a6e5c7dcefe42,09319257834,351,7,Quidem veritatis quis unde sint magni laudanti...,http://www.lowemurray.org/
